## Preparing necessary libraries

In [109]:
import numpy as np 
import warnings
from qiskit import QuantumCircuit, transpile
from qiskit.primitives.sampler import Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
warnings.filterwarnings('ignore')

## Preparing Quantum Circuit

In [110]:
qc = QuantumCircuit(3)

In [111]:
qc.h(0) # Superposition (Hadamard Gate)
qc.p(np.pi/2,0) # Quantum Phase (Phase Gate)
qc.cx(0,1) # 0th-qubit-Controlle-NOT gate on 1st (CNOT)
qc.cx(0,2) # 0th-qubit-Controlle-NOT gate on 1st

## Drawing the circuit

In [112]:
qc.draw()

┌───┐┌────────┐          
q_0: ┤ H ├┤ P(π/2) ├──■────■──
     └───┘└────────┘┌─┴─┐  │  
q_1: ───────────────┤ X ├──┼──
                    └───┘┌─┴─┐
q_2: ────────────────────┤ X ├
                         └───┘

## Measurement of all

In [113]:
qc_measured = qc.measure_all(inplace=False)

## Sampler

In [114]:
sampler = Sampler()
job = sampler.run(qc_measured, shots=1000000)
result = job.result()
print(f" > Quasi probability distribution: {result.quasi_dists}") # 000 50 % / 111 50 %

 > Quasi probability distribution: [{0: 0.499499, 7: 0.500501}]


## Defining the observable

In [115]:
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

## Estimator

In [116]:
estimator = Estimator()
job = estimator.run(qc, operator, shots=1000)
result = job.result()
print(f" > Expectation values: {result.values}")

 > Expectation values: [4.]


## Transpile

In [117]:
qc_transpiled = transpile(qc, basis_gates = ['cz', 'sx', 'rz'], coupling_map =[[0, 1], [1, 2]] , optimization_level=3)